# Final JSteg Blocks — True DCT Steganography
## Replace Block 3, 4, 5 in your main notebook with these
## Also replace Block 7 and add Block 5b after Block 5
This works directly in JPEG compressed domain — no pixel conversion, no rounding errors.

In [ ]:
# ============================================================
# BLOCK 3 (Final JSteg): Helpers only — no DCT math needed
# because jpegio gives us DCT coefficients directly
# ============================================================
import numpy as np
import cv2
import jpegio
import os

# We skip these DCT coefficient values during embedding.
# 0  -> JPEG often discards zeros, unreliable
# 1  -> too close to zero, JPEG may round to 0
# -1 -> same reason
SKIP_VALUES = {0, 1, -1}


def text_to_bits(text):
    """Convert string to flat list of bits, 8 bits per character."""
    bits = []
    for char in text:
        byte = ord(char)
        for i in range(7, -1, -1):
            bits.append((byte >> i) & 1)
    return bits


def bits_to_text(bits):
    """Convert flat list of bits back to string."""
    chars = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) < 8:
            break
        value = int(''.join(str(b) for b in byte), 2)
        chars.append(chr(value))
    return ''.join(chars)


print('Block 3 Final JSteg ready.')
print('jpegio imported successfully — working in true JPEG DCT domain.')

In [ ]:
# ============================================================
# BLOCK 4 (Final JSteg): Embed directly into JPEG DCT coefficients
# ============================================================

def embed_message(image_path, message, output_path):
    """
    Hide a secret message inside an image using true JSteg.

    HOW IT WORKS:
    Instead of going image -> pixels -> DCT -> embed -> IDCT -> pixels -> save
    (which loses data due to uint8 rounding), we:
    1. Save the cover image as JPEG first
    2. Read the JPEG's compressed DCT coefficients DIRECTLY using jpegio
    3. Modify the LSB (least significant bit) of each coefficient
       to encode our message bits — this is exactly JSteg
    4. Write the modified coefficients back to a new JPEG file

    WHY this survives JPEG re-compression:
    We are already working INSIDE the JPEG compressed domain.
    The coefficients we modify ARE the JPEG coefficients.
    Re-saving as JPEG does not change them again.

    NOTE: Input can be PNG or JPEG. Output is always JPEG (stego image).
    The output_path should end with .jpg
    """

    # --- Step 1: Convert cover image to JPEG so jpegio can read it ---
    # If cover is PNG, we must save it as JPEG first
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f'Cannot open: {image_path}')

    # Save as high quality JPEG temporarily
    temp_cover_jpg = '_temp_cover.jpg'
    cv2.imwrite(temp_cover_jpg, img, [cv2.IMWRITE_JPEG_QUALITY, 95])

    # --- Step 2: Read JPEG DCT coefficients directly ---
    # jpegio.read() gives us the raw integer DCT coefficients
    # exactly as stored in the JPEG file — no IDCT, no pixel values
    jpeg_struct = jpegio.read(temp_cover_jpg)

    # Get the luminance (Y) channel coefficients — channel index 0
    # Shape: (height, width) where values are raw DCT integers
    coeff_matrix = jpeg_struct.coef_arrays[0].copy()
    print(f'DCT coefficient matrix shape: {coeff_matrix.shape}')
    print(f'Coefficient value range: {coeff_matrix.min()} to {coeff_matrix.max()}')

    # --- Step 3: Prepare message bits ---
    full_message = message + '\x00'  # null terminator marks end
    bits = text_to_bits(full_message)
    total_bits = len(bits)

    # Count usable coefficients (skip zeros and ±1)
    usable = np.sum(~np.isin(coeff_matrix, list(SKIP_VALUES)))
    print(f'Usable coefficients : {usable}')
    print(f'Bits to embed       : {total_bits}')
    print(f'Max characters      : ~{usable // 8}')

    if total_bits > usable:
        raise ValueError(f'Message too long! Max ~{usable // 8} characters.')

    # --- Step 4: Embed bits into LSB of DCT coefficients ---
    # JSteg: for each non-zero, non-±1 coefficient:
    #   set its last bit (LSB) to our message bit
    #   bit=1 -> make coefficient odd
    #   bit=0 -> make coefficient even
    bit_index = 0
    rows, cols = coeff_matrix.shape

    for r in range(rows):
        for c in range(cols):
            if bit_index >= total_bits:
                break

            val = int(coeff_matrix[r, c])

            # Skip unreliable coefficients
            if val in SKIP_VALUES:
                continue

            # Get the bit we want to embed
            bit = bits[bit_index]

            # Current LSB of this coefficient
            current_lsb = abs(val) % 2

            if current_lsb != bit:
                # Flip the LSB by adding or subtracting 1
                # Keep the sign of the coefficient unchanged
                if val > 0:
                    val += 1
                else:
                    val -= 1

            coeff_matrix[r, c] = val
            bit_index += 1

    # --- Step 5: Write modified coefficients back to JPEG file ---
    jpeg_struct.coef_arrays[0] = coeff_matrix

    # Ensure output path ends with .jpg
    if not output_path.lower().endswith('.jpg'):
        output_path = output_path.replace('.png', '.jpg')
        print(f'Note: output changed to {output_path} (must be JPEG)')

    jpegio.write(jpeg_struct, output_path)
    os.remove(temp_cover_jpg)

    print(f'\nStego JPEG saved: {output_path}')
    print(f'Embedded {bit_index} bits = {bit_index // 8} characters')

    # Return as numpy array for display compatibility
    return cv2.imread(output_path)


print('embed_message() Final JSteg defined.')

In [ ]:
# ============================================================
# BLOCK 5 (Final JSteg): Extract directly from JPEG DCT coefficients
# ============================================================

def extract_message(stego_path):
    """
    Extract hidden message from stego JPEG.
    Reads DCT coefficients directly — same skip logic as embedding.
    Works even after JPEG re-compression because we are in JPEG domain.
    """

    # If given a PNG, convert to JPEG first so jpegio can read it
    if stego_path.lower().endswith('.png'):
        temp_jpg = '_temp_extract.jpg'
        img = cv2.imread(stego_path)
        cv2.imwrite(temp_jpg, img, [cv2.IMWRITE_JPEG_QUALITY, 95])
        read_path = temp_jpg
    else:
        read_path = stego_path
        temp_jpg = None

    jpeg_struct   = jpegio.read(read_path)
    coeff_matrix  = jpeg_struct.coef_arrays[0]

    if temp_jpg and os.path.exists(temp_jpg):
        os.remove(temp_jpg)

    bits = []
    rows, cols = coeff_matrix.shape

    for r in range(rows):
        for c in range(cols):
            val = int(coeff_matrix[r, c])

            # Same skip logic as embedding — must match exactly
            if val in SKIP_VALUES:
                continue

            # Extract LSB
            bit = abs(val) % 2
            bits.append(bit)

            # Check for null terminator every 8 bits
            if len(bits) % 8 == 0:
                last_byte = int(''.join(str(b) for b in bits[-8:]), 2)
                if last_byte == 0:  # end of message
                    chars = []
                    for i in range(0, len(bits) - 8, 8):
                        val2 = int(''.join(str(b) for b in bits[i:i+8]), 2)
                        chars.append(chr(val2))
                    return ''.join(chars)

    return bits_to_text(bits)


print('extract_message() Final JSteg defined.')

In [ ]:
# ============================================================
# BLOCK 7 (Updated): Set inputs and embed
# NOTE: STEGO_IMAGE_PATH must now end with .jpg not .png
# ============================================================

COVER_IMAGE_PATH = 'your_image.png'          # <-- your cover image (PNG or JPEG)
SECRET_MESSAGE   = 'Hello, this is a secret message hidden with DCT and i am shoyo trying to hide that from must'
STEGO_IMAGE_PATH = 'stego_image.jpg'         # <-- output MUST be .jpg now

stego = embed_message(
    image_path  = COVER_IMAGE_PATH,
    message     = SECRET_MESSAGE,
    output_path = STEGO_IMAGE_PATH
)

print('\nEmbedding complete!')

In [ ]:
# ============================================================
# BLOCK 5b: JPEG survival test
# Run AFTER Block 7 to verify message survives re-compression
# ============================================================
import os

print('Testing JPEG survival at different quality levels...\n')
print(f'Original: "{SECRET_MESSAGE}"\n')
print(f'{"Quality":<12} {"Survived":<12} {"Corrupted":<15} {"Extracted preview"}')
print('-' * 80)

for quality in [95, 90, 75, 50]:
    recompressed = f'test_recompressed_q{quality}.jpg'

    # Re-compress the stego JPEG at this quality level
    img = cv2.imread(STEGO_IMAGE_PATH)
    cv2.imwrite(recompressed, img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # Extract from the re-compressed image
    extracted = extract_message(recompressed)
    survived  = (extracted == SECRET_MESSAGE)
    corrupted = sum(1 for a, b in zip(SECRET_MESSAGE, extracted) if a != b)
    corrupted += abs(len(SECRET_MESSAGE) - len(extracted))

    status   = 'YES ✓' if survived else 'NO  ✗'
    c_str    = f'{corrupted}/{len(SECRET_MESSAGE)}'
    preview  = extracted[:45] + '...' if len(extracted) > 45 else extracted
    print(f'Q={quality:<9} {status:<12} {c_str:<15} "{preview}"')

    os.remove(recompressed)